# Project : 3D House (Flanders)

## For given address in Flanders need to plot the 3D House/building

### Given the DSM, DTM tif files for flanders splitted in smaller tif zipped and stored on url

DSM : http://www.geopunt.be/download?container=dhm-vlaanderen-ii-dsm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DSM,%20raster,%201m

DTM:http://www.geopunt.be/download?container=dhm-vlaanderen-ii-dtm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DTM,%20raster,%201m


# Importing the packages required

In [72]:
import gdal
import rasterio as rio
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib import pyplot
from osgeo import gdal
import numpy as np
import requests
import json
import fiona
import geopy
import pyproj
import shapely
import osr
import os
import pygis
import richdem as rd
%matplotlib inline

##  Loading all zip tif files from url 

In [73]:
downloadurl = 'http://www.geopunt.be/download?container=dhm-vlaanderen-ii-dsm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DSM,%20raster,%201m'

#  Coding for a rendering 3d house from tiff file

## Work Flow

In [76]:
# 1 Get the address from the user
# 2 Extract lat_lon from address by using lam_lat_lon function
# 3 For all the zipped tif DTM & DSM on url form df of metadata.
# 4 Form a df for all tiled tif files using tif_metadata function
# 5 Find the tile in which the given co-oridinate is present
# 6 3D of the location

# Sample Address:Francis Wellesplein 1, 2018 Antwerpen (152074.52, 210653.91)
# Sint-Pietersvliet 3, 2000 Antwerpen
# Tavernierkaai 3, 2000 Antwerpen

### Address to co-ordiante 

In [83]:
address = input("Enter the Address as 'street_name', 'number', 'postcode', 'city' : ")


Enter the Address as 'street_name', 'number', 'postcode', 'city' : Francis Wellesplein 1, 2018 Antwerpen


In [86]:
def lam_lat_lon(address):
    while True:
        try:
            req = requests.get(f"http://loc.geopunt.be/geolocation/location?q={address}&c=1",)
            x = req.json()["LocationResult"][0]["Location"]["X_Lambert72"]
            y = req.json()["LocationResult"][0]["Location"]["Y_Lambert72"]
            return x, y
            continue

        except IndexError: 
            print("Entered address is not in correct format please retry") 
            break

given_loc = lam_lat_lon(address)
print(type(given_loc))
print(given_loc)

<class 'tuple'>
(152074.52, 210653.91)


###  Function for metadata of Bigger tif files

In [93]:
# Defination for storing all the information about tile.

in_dsm_path = 'D:/BeCode/Maps/Test/DSM_Files/' 
in_dsm_file = os.listdir(in_path) 

def tif_metadata(infile):
    
    ds = gdal.Open(infile)  #'path/to/file'
    my_file = rio.open(infile)
    
    width = ds.RasterXSize
    height = ds.RasterYSize
    gt = ds.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5] 
    maxx = gt[0] + width*gt[1] + height*gt[2]
    maxy = gt[3]     
    center = my_file.xy((my_file.height // 2), (my_file.width // 2))    
   
    return [width, height, gt, minx, miny, maxx, maxy, center]


tiffinfo = tif_metadata(in_dsm_path + in_dsm_file[0])
print(tiffinfo)

width = tiffinfo[0]
print('width', width)
height = tiffinfo[1]
print('height', height)
gt = tiffinfo[2]
print('gt', gt)
minx = tiffinfo[3]
print('minx', minx)
miny = tiffinfo[4]
print('miny', miny)
maxx = tiffinfo[5]
print('maxx', maxx)
maxy = tiffinfo[6]
print('maxy', maxy)
print("Center of tiff", tiffinfo[7])

[17000, 9000, (145000.0, 1.0, 0.0, 247000.0, 0.0, -1.0), 145000.0, 238000.0, 162000.0, 247000.0, (153500.5, 242499.5)]
width 17000
height 9000
gt (145000.0, 1.0, 0.0, 247000.0, 0.0, -1.0)
minx 145000.0
miny 238000.0
maxx 162000.0
maxy 247000.0
Center of tiff (153500.5, 242499.5)


### Splitting big tif to smaller chunks using GDAL

In [40]:
# File name contains the 

in_path = 'D:/BeCode/Maps/Test/DSM_Files/'            #Input file path 
input_filename = 'DHMVIIDSMRAS1m_k15.tif'             # Input tif

out_path = 'D:/BeCode/Maps/Test/Tiled_DSM_Output/'    #Tile files path
output_filename = ''                                  # Output tif 

tile_size_x = 1000
tile_size_y = 1000

ds = gdal.Open(in_path + input_filename)
band = ds.GetRasterBand(1)
xsize = band.XSize  
ysize = band.YSize

gt = ds.GetGeoTransform()
minx = gt[0]
maxy = gt[3] 

for i in range(0, xsize, tile_size_x):     
    for j in range(0, ysize, tile_size_y):        
        com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(in_path) + str(input_filename) + " " + str(out_path) + str(output_filename) + str(i) + "_" + str(j) + "_namestart_" + str(minx+(tile_size_x/2)+i) + "_" + str(maxy-(tile_size_y/2)-j) + ".tif"
        os.system(com_string)


### Prepare dataframe for small tile tif files.

In [41]:
out_path = 'D:/BeCode/Maps/Test/Tiled_DSM_Output/'

files = os.listdir(out_path)
namelist =[]
for f in files:    
    namelist.append(f)

df = pd.DataFrame(namelist)
df[['1','2','3','x_center','y_center']] = df[0].str.split('_',expand=True)
df['y_center'] = df['y_center'].str.replace('.tif','')
df['x_center'] = pd.to_numeric(df['x_center'], errors='ignore') #Str to int
df['y_center'] = pd.to_numeric(df['y_center'], errors='ignore') #Str to int

print(df.head())
print(df.shape)

                                         0  1      2          3  x_center  \
0      0_0_namestart_130500.0_217500.0.tif  0      0  namestart  130500.0   
1      0_0_namestart_145500.0_246500.0.tif  0      0  namestart  145500.0   
2      0_0_namestart_226500.0_226500.0.tif  0      0  namestart  226500.0   
3  0_10000_namestart_130500.0_207500.0.tif  0  10000  namestart  130500.0   
4   0_1000_namestart_130500.0_216500.0.tif  0   1000  namestart  130500.0   

   y_center  
0  217500.0  
1  246500.0  
2  226500.0  
3  207500.0  
4  216500.0  
(902, 6)


In [42]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x_center, df.y_center))
print(gdf.head())

                                         0  1      2          3  x_center  \
0      0_0_namestart_130500.0_217500.0.tif  0      0  namestart  130500.0   
1      0_0_namestart_145500.0_246500.0.tif  0      0  namestart  145500.0   
2      0_0_namestart_226500.0_226500.0.tif  0      0  namestart  226500.0   
3  0_10000_namestart_130500.0_207500.0.tif  0  10000  namestart  130500.0   
4   0_1000_namestart_130500.0_216500.0.tif  0   1000  namestart  130500.0   

   y_center                       geometry  
0  217500.0  POINT (130500.000 217500.000)  
1  246500.0  POINT (145500.000 246500.000)  
2  226500.0  POINT (226500.000 226500.000)  
3  207500.0  POINT (130500.000 207500.000)  
4  216500.0  POINT (130500.000 216500.000)  


### Nearest tile center from geo data frame

In [43]:

from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

def tile_finder():
    
    pts = gdf['geometry']
    pts =  pts.unary_union # convert to Shapely MultiPoint geometry
    pt = Point(lam_lat_lon(address))   # Tuple to Point conversion
    output = [o.wkt for o in nearest_points(pt, pts)]
        
    return output[1]

tile_finder()

'POINT (151500 211500)'

### Identify the tile name for the given_loc

In [46]:
# Identify the file name containg the tile.

out_path = 'D:/BeCode/Maps/Test/Tiled_DSM_output/'
rawinfile = tile_finder()                           # Tile center point 
#print(rawinfile)
sp = rawinfile.split(" ")
fn = (sp[1])[1:] + ".0_" + (sp[2][:-1]) + ".0"
tile_filename = [d for d in os.listdir(out_path) if fn in d]
print(tile_filename)

['21000_6000_namestart_151500.0_211500.0.tif']


In [48]:
tile_dsmfile = 'D:/BeCode/Maps/Test/Tiled_DSM_Output/' + str(tile_filename[0])
tile_dtmfile = 'D:/BeCode/Maps/Test/Tiled_DTM_Output/' + str(tile_filename[0])

print(tile_dtmfile)
print(tile_dsmfile)


D:/BeCode/Maps/Test/Tiled_DTM_Output/21000_6000_namestart_151500.0_211500.0.tif
D:/BeCode/Maps/Test/Tiled_DSM_Output/21000_6000_namestart_151500.0_211500.0.tif


In [49]:
# API for address = https: //https://api.basisregisters.dev-vlaanderen.be/v1/adresmatch {objectId}
# API for Building = https://api.basisregisters.dev-vlaanderen.be/v1/gebouweenheden/ {objectId} 
# API for polygon =https://api.basisregisters.dev-vlaanderen.be/v1/gebouwen/
#address = (Leopold de Waelplaats 2, 2000 Antwerpen (151757.75, 211106.12))

import regex as re

address_regx = re.compile("^([A-z- ]+)\s(\d+),\s(\d+)\s([A-z]+)")
result = address_regx.search(address)  #Address from user
street = result.group(1)
nb = result.group(2)
pc = result.group(3)
city = result.group(4)

req1 = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/adresmatch?gemeentenaam={city}&straatnaam={street}&huisnummer={nb}&postcode={pc}").json()
objectId = req1["adresMatches"][0]["adresseerbareObjecten"][0]["objectId"]

req2 = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouweenheden/{objectId}").json()
objectId = req2["gebouw"]["objectId"]

req3 = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouwen/{objectId}").json()
polygon = [req3["geometriePolygoon"]["polygon"]]
#polygon


In [94]:
from rasterio.mask import mask

tile_data = rio.open(tile_dsmfile)

crop_img, crop_transform = mask(dataset=tile_data, shapes=polygon, crop=True, indexes=1, nodata=0, filled=True)
#print(crop_img)

fig = go.Figure(data=[go.Surface(z=crop_img)])

fig.update_layout(title='3D Plot', autosize=True)

fig.show()